In [35]:
import csv
import numpy as np
import io
import sys

In [36]:
from google.colab import files
uploaded = files.upload()

Saving transaction1.csv to transaction1 (1).csv
Saving transaction2.csv to transaction2 (1).csv
Saving transaction3.csv to transaction3 (1).csv
Saving transaction4.csv to transaction4 (1).csv
Saving transaction5.csv to transaction5 (1).csv


In [76]:
# defining all the input transactions as the input database

def load_database(transaction): 

    with open(transaction,'r') as transaction_csv:

      #Read a comma-separated values (csv) file into DataFrame.
        read_csv = csv.reader(transaction_csv, delimiter=',')
        next(read_csv)
        rows = []
      

        #add rows at the end to the existing dataframe.
        for row in read_csv:
            rows.append(row[1])

        #Split the data by using the , separator
        for row in transaction_csv:
          str_line=list(lines.strip().split(","))
          _t=list(np.unique(str_line))
          #item order in the list
          _t.sorted(key=lambda x:order.index(x))

    return rows 


In [83]:
#Function to find out all the possible subsets of the data for each of the dataset.
def subsets(s):
    length_s = len(s)
    possible_subsets = []
    #Bitwise operation to check boolean values.
    for i in range(1 << length_s):      
        possible_subsets.append([s[j] for j in range(length_s) if (i & (1 << j))])       
    return possible_subsets
    print()

#Function to calculate the support for each of the items in the item set. Support(A) = (Transactions containing (A))/(Total Transactions)
def support_calculated(data, items):
    support = 0
    for row in data:
        if set(items).issubset(row):
            support += 1
    return support
    print()

In [84]:
#Function to find the association between the frequent items
def association1(data, freq_items, dist_items, min_conf):
    alist = []
    for item in freq_items:
        powerset = subsets(item)
        for p in powerset:
            l = set(p)
            r = set(item) - l
            if len(l) and len(r):
                support_lhs = support_calculated(data, l)
                if support_lhs == 0:
                    continue
                confidence = support_calculated(data, item) * 100 / support_lhs
                if confidence < min_conf:
                    continue
                lhs_items = [dist_items[i] for i in l]
                rhs_items = [dist_items[i] for i in r]
                alist.append((lhs_items, rhs_items, confidence))
    return alist

In [85]:
#Function to compare the support to that of the new frequent items in the dataset.
def compare(data, elements, min_support):
    frequent_items = []
    non_frequent_items = []
    if elements is not None:
        for e in elements:
            if(support_calculated(data, e) >= min_support):
                frequent_items.append(e)
            else:
                non_frequent_items.append(e)
    return frequent_items, non_frequent_items


def new(pos_freq_elements, neg_freq_elements):
    new = {}
    total_pos_items = len(pos_freq_elements)
    if total_pos_items == 0:
        return []
    len_each_item = len(pos_freq_elements[0])
    for left in range(0, total_pos_items):
        for right in range(left+1, total_pos_items):
            merged = tuple(sorted(set(pos_freq_elements[left]).union(set(pos_freq_elements[right]))))
            if len(merged) == len_each_item + 1 and not check_non_frequent_item(merged, neg_freq_elements):
                new[merged] = 1
    return [list(i) for i in new.keys()]
def check_non_frequent_item(element, non_freq_elements):
    all_pos = subsets(element)
    for item in all_pos:
        if item in non_freq_elements:
            return True
    return False

In [86]:
def unique_items(rows):
    items_in_row = []
    for row in rows:
        items_in_row.extend(row.split(","))
        
    return list(set(items_in_row))

def unique_val(all_rows, dst_its):
    rows = []
    for row in all_rows:
        rows.append(sorted([dst_its.index(item) for item in row.split(",")]))
    return rows

def association_rule(transaction, min_support, min_conf):
    all_rows = load_database(transaction) # load_database()
    dist_items = unique_items(all_rows)
    num_distinct_items = len(dist_items)
    data = unique_val(all_rows, dist_items)
    
    freq_items = apriori_algorithm(data, num_distinct_items, min_support, min_conf)
    alist = association1(data, freq_items, dist_items, min_conf)
    print("Total number of items in association list is {}".format(len(alist)))
    for item in alist:
        lhs_items, rhs_items, confidence = item
        print("{} -->> {} : {}".format(lhs_items, rhs_items, confidence))

In [87]:
def apriori_algorithm(data, item_size, min_support, min_conf):
    freq_items = []
    non_freq_items = []
    items = range(0, item_size)
    min_support_value = int(len(data) * min_support / 100.0)
    new_freq_elements = [[i] for i in items]
    pos_freq_elements, neg_freq_elements = compare(data, new_freq_elements, min_support_value)
    freq_items.extend(pos_freq_elements)
    non_freq_items.extend(neg_freq_elements)
        
    for k in range(2,item_size+1):
        new_freq_elements = new(pos_freq_elements, non_freq_items)
        pos_freq_elements, neg_freq_elements = compare(data, new_freq_elements, min_support_value)
        if not pos_freq_elements:
            break
        freq_items.extend(pos_freq_elements)
        non_freq_items.extend(neg_freq_elements)
    
    return freq_items

In [92]:
print("For transactional database1 ")
transaction = 'transaction1.csv'
min_support = input("Enter support: ")
min_conf = input("Enter Confidence: ")
print("Association Rules :")
association_rule(transaction, float(min_support), float(min_conf))


For transactional database1 
Enter support: 15
Enter Confidence: 75
Association Rules :
Total number of items in association list is 60
['Meat'] -->> [' Fruits'] : 75.0
[' Cheese'] -->> [' Fruits'] : 83.33333333333333
['Cheese'] -->> [' Vegetables'] : 100.0
['Meat'] -->> [' Dairy'] : 75.0
[' Snacks'] -->> [' Dairy'] : 75.0
[' Eggs'] -->> [' Dairy'] : 100.0
['Meat'] -->> [' Vegetables'] : 75.0
['Meat'] -->> [' Cheese'] : 75.0
['Meat'] -->> [' Snacks'] : 75.0
['Snacks'] -->> [' Beverage'] : 100.0
[' Seafood'] -->> [' Vegetables'] : 80.0
['Meat'] -->> [' Fruits', ' Dairy'] : 75.0
[' Fruits', 'Meat'] -->> [' Dairy'] : 100.0
[' Dairy', 'Meat'] -->> [' Fruits'] : 100.0
[' Fruits', ' Vegetables'] -->> [' Dairy'] : 80.0
[' Fruits', ' Cheese'] -->> [' Dairy'] : 80.0
[' Dairy', ' Cheese'] -->> [' Fruits'] : 100.0
[' Fruits', ' Eggs'] -->> [' Dairy'] : 100.0
['Meat'] -->> [' Fruits', ' Snacks'] : 75.0
[' Fruits', 'Meat'] -->> [' Snacks'] : 100.0
[' Snacks', 'Meat'] -->> [' Fruits'] : 100.0
[' Fru

In [90]:
print("For transactional database2 ")
transaction = 'transaction2.csv'
min_support = input("Enter the minimum support: ")
min_conf = input("Enter the minimum Confidence: ")
print("Association Rules :")
association_rule(transaction, float(min_support), float(min_conf))
print()

For transactional database2 
Enter the minimum support: 10
Enter the minimum Confidence: 10
Association Rules :
Total number of items in association list is 106
[' crayons'] -->> [' pen'] : 28.571428571428573
[' pen'] -->> [' crayons'] : 20.0
[' crayons'] -->> ['clip'] : 28.571428571428573
['clip'] -->> [' crayons'] : 40.0
[' crayons'] -->> [' eraser'] : 57.142857142857146
[' eraser'] -->> [' crayons'] : 57.142857142857146
[' crayons'] -->> [' sharpner'] : 71.42857142857143
[' sharpner'] -->> [' crayons'] : 55.55555555555556
[' crayons'] -->> [' pin'] : 28.571428571428573
[' pin'] -->> [' crayons'] : 25.0
[' crayons'] -->> [' book'] : 28.571428571428573
[' book'] -->> [' crayons'] : 40.0
[' ruler'] -->> [' pen'] : 60.0
[' pen'] -->> [' ruler'] : 30.0
[' ruler'] -->> ['eraser'] : 80.0
['eraser'] -->> [' ruler'] : 80.0
[' ruler'] -->> [' pencil'] : 40.0
[' pencil'] -->> [' ruler'] : 33.333333333333336
['tape'] -->> [' book'] : 100.0
[' book'] -->> ['tape'] : 40.0
[' pen'] -->> ['clip'] :

In [93]:
print("For transactional database3 ")
transaction = 'transaction3.csv'
min_support = input("Enter the minimum support: ")
min_conf = input("Enter the minimum Confidence: ")
print("Association Rules :")
association_rule(transaction, float(min_support), float(min_conf))

For transactional database3 
Enter the minimum support: 50
Enter the minimum Confidence: 75
Association Rules :
Total number of items in association list is 0


In [94]:
print("For transactional database4 ")
transaction = 'transaction4.csv'
min_support = input("Enter the minimum support: ")
min_conf = input("Enter the minimum Confidence: ")
print("Association Rules :")
association_rule(transaction, float(min_support), float(min_conf))

For transactional database4 
Enter the minimum support: 50
Enter the minimum Confidence: 75
Association Rules :
Total number of items in association list is 2
[' TV stand'] -->> [' chair'] : 76.92307692307692
[' coffee table'] -->> [' chair'] : 78.57142857142857


In [96]:
print("For transactional database5 ")
transaction = 'transaction5.csv'
min_support = input("Enter the minimum support: ")
min_conf = input("Enter the minimum Confidence: ")
print("Association Rules :")
association_rule(transaction, float(min_support), float(min_conf))


For transactional database5 
Enter the minimum support: 25
Enter the minimum Confidence: 40
Association Rules :
Total number of items in association list is 14
[' dress'] -->> [' jacket '] : 50.0
[' jacket '] -->> [' dress'] : 83.33333333333333
[' dress'] -->> [' jeans'] : 50.0
[' jeans'] -->> [' dress'] : 62.5
[' dress'] -->> [' sweatshirt'] : 60.0
[' sweatshirt'] -->> [' dress'] : 85.71428571428571
[' jacket '] -->> [' sweatshirt'] : 83.33333333333333
[' sweatshirt'] -->> [' jacket '] : 71.42857142857143
[' dress'] -->> [' jacket ', ' sweatshirt'] : 50.0
[' jacket '] -->> [' dress', ' sweatshirt'] : 83.33333333333333
[' dress', ' jacket '] -->> [' sweatshirt'] : 100.0
[' sweatshirt'] -->> [' dress', ' jacket '] : 71.42857142857143
[' dress', ' sweatshirt'] -->> [' jacket '] : 83.33333333333333
[' jacket ', ' sweatshirt'] -->> [' dress'] : 100.0
